In [ ]:
using HDF5,PyPlot,Cairo, Images, Colors


In [ ]:
println("kernel is working")

In [ ]:
################ load image array from h5 file ##########
#########################################################
path = "/mnt/data/rolilab/rawFiles_2/viscosityProject"
folders = readdir(path)
count = length(readdir(path*"/"*folders[1]))
#ASSUMING THAT EACH pvp% HAS THE SAME # OF TRIALS, count
subfolders = fill("",4,count)
foldersPath = fill("",4)
subfoldersPath = fill("",4,count)
fileNames = fill("",4,count)
filePaths = fill("",4,count)

for i =  1:length(folders)
    #i = PVP%
    foldersPath[i] = (path*"/"*folders[i])
    foo = readdir(foldersPath[i])
    for j = 1:length(subfolders[1,:])
        #j = trial number
        subfolders[i,j] = foo[j]
        subfoldersPath[i,j] = foldersPath[i]*"/"*subfolders[i,j]
        fileNames[i,j] = readdir(subfoldersPath[i,j])[1]
        filePaths[i,j] = subfoldersPath[i,j]*"/"*fileNames[i,j]
    end
    
end
println("All files saved")

#file = h5open(mainFolder*"/"*filename,"r")
#image = read(file["images"])
#size(image)

In [ ]:
################## display movie #########################
###########################################################
# step = 1
# startframe = 580
# endframe = 700

# for i = startframe:step:endframe
# i =604
# #transform frames into UInt8 
#     img = round(UInt8,image(i)*1) 
#    img = image[:,:,i]
#    imgRgb = cat(3,img',img',img')
#    display(colorim(imgRgb[:,:,:])); 
#      println(i)
#     IJulia.clear_output(true)
# end


In [ ]:
################## store maximum index #############
#######################################################
maxValues = Array{Int64}(4,count)
for i =  1:length(folders)       
    for j = 1:length(subfolders[1,:])
        file = h5open(filePaths[i,j],"r")
        image = read(file["images"])        
        sizeOfObj = zeros(1, size(image,3))
        for n = 500:700

            img1 = float(image[:,:,n-1])
            img2 = float(image[:,:,n])


            # img1 = float(image[:,:,600])
            #  img2 = float(image[:,:,601])

            imgSub = (img2 - img1)


            indThresd = find(imgSub .> 10)

            imgSub2 = zeros(UInt8, size(imgSub,1),size(imgSub,2))
            #display(imgSub2)
            imgSub2[indThresd] = 255

                sizeOfObj[1,n] = length(indThresd)

            # #######rid of negative values#######

            # for j = 1:1:


            # end

            # imgRgb = cat(3,imgSub2',imgSub2',imgSub2')
            #  imgRgb = cat(3,img1',img1',img1')
            #  imgRgb = cat(3,img2',img2',img2')

            #    display(colorim(imgRgb[:,:,:])); 

            #     println(n)
            #     IJulia.clear_output(true)

       end
        maxValues[i,j] = indmax(sizeOfObj)
    end
end
#################### SHOW MAX VALUE MATRIX #################################################
###########################################################################
display(maxValues)

In [ ]:
#################### PLOT #################################################
###########################################################################


# plot(sizeOfObj', "k.")


# indmax(sizeOfObj)

In [ ]:
#################### save H5 file in save folder with same name ###########
###########################################################################
for i =  1:length(folders)       
    for j = 1:length(subfolders[1,:])
        h5open(subfoldersPath[i,j]*"/"*"TIME"*fileNames[i,j][5:end], "w") do file
            data = h5open(filePaths[i,j],"r")
            image = read(data["images"])
            write(file, "timeOfInjection", maxValues[i,j])
            write(file, "PVP%", i-1)
            f = maxValues[i,j]
            write(file, "injectionFrame", image[:,:,f])
        end
    end
end

In [ ]:
for i =  1:length(folders)       
    for j = 1:length(subfolders[1,:])
        c = h5open(subfoldersPath[i,j]*"/"*"TIME"*fileNames[i,j][5:end], "r") do file
            image = read(file["injectionFrame"])
            imgRgb = cat(3,image',image',image')
            println(read(file, "timeOfInjection"))
            display(colorim(imgRgb[:,:,:])); 
        end
    end
end